In [1]:
# Import Azure ML API SDK. The SDK is installed implicitly with the latest
# version of the CLI in your default python environment
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

In [2]:
#test_image_url = "https://s7d1.scene7.com/is/image/MoosejawMB/10184925x1011463_zm?$thumb225$"
#test_image_url = "http://www.ems.com/dw/image/v2/AAQU_PRD/on/demandware.static/-/Sites-vestis-master-catalog/default/dwd70784cc/product/images/2010/536/2010536/2010536_601_main.jpg?sw=195&sh=195&sm=fit"
test_image_url = "https://ec-i21.geccdn.net/site/images/n-picgroup/DBA_098-2000112.jpg"
#test_image_url = "https://cdn.shopify.com/s/files/1/1803/7259/files/hammock-rope-carabiner_large.jpg?418721957162400510"


In [3]:
%%writefile score.py

import numpy as np
import tensorflow as tf

from PIL import Image
import requests

label_file = "/home/hacker1/source/tfmodel/output_labels.txt"
model_file = "/home/hacker1/source/tfmodel/output_graph.pb"

def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def read_tensor_from_image_file(url, input_height=299, input_width=299,input_mean=0, input_std=255):
    im = Image.open(requests.get(url, stream=True).raw)
    
    im.resize((input_height,input_width), Image.ANTIALIAS)
    image_array = np.array(im)[:, :, 0:3]  # Select RGB channels only.
    
    float_caster = tf.cast(image_array, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0);
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)

    return result

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

def init():   
    # read in the model file
    global graph
    graph = load_graph(model_file)

def run(url):
    try:
        input_layer = "Mul"
        output_layer = "final_result"
        input_name = "import/" + input_layer
        output_name = "import/" + output_layer
        input_operation = graph.get_operation_by_name(input_name)
        output_operation = graph.get_operation_by_name(output_name)
        
        t= read_tensor_from_image_file(url)

        with tf.Session(graph=graph) as sess:
            results = sess.run(output_operation.outputs[0], {
                input_operation.outputs[0]: t
            })
            results = np.squeeze(results)

        top_k = results.argsort()[-5:][::-1]
        labels = load_labels(label_file)
        #for i in top_k:
        return(str(labels[top_k[0]])+" "+str(results[top_k[0]]))
    except Exception as e:
        return (str(e))

Overwriting score.py


In [4]:
import score
score.init()
score.run(test_image_url)
#print(type(test_image_url))

'carabiners 0.999843'

In [5]:
# Define the input dataframe and create the schema
url = "https://ec-i21.geccdn.net/site/images/n-picgroup/DBA_098-2000112.jpg"
inputs = {"url": SampleDefinition(DataTypes.STANDARD, url)}
generate_schema(run_func=score.run, inputs=inputs, filepath='/home/hacker1/tfmodel/service_schema.json')

{'input': {'url': {'internal': 'gANjYXp1cmVtbC5hcGkuc2NoZW1hLnB5dGhvblV0aWwKUHl0aG9uU2NoZW1hCnEAKYFxAX1xAlgJAAAAZGF0YV90eXBlcQNjYnVpbHRpbnMKc3RyCnEEc2Iu',
   'swagger': {'example': 'https://ec-i21.geccdn.net/site/images/n-picgroup/DBA_098-2000112.jpg',
    'type': 'string'},
   'type': 0,
   'version': '1.0.1b6'}}}